# Omar Farooq
## BIG DATA ANALYTICS using Spark
### ASSIGNMENT 2

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

In [2]:
# create the session
conf = pyspark.SparkConf().setAppName('app').setMaster('local')
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
spark

##### Determine the number of long comments given by each users where the length of the long comment should be greater than 20 alphabets.

In [ ]:
Data = sc.textFile("log1.txt")

In [25]:
comments = Data.map(lambda line: line.split(','))
lngcmnts = comments.filter(lambda x: len(x[1])>20)
cmntmap = lngcmnts.map(lambda line: (line[0], 1))
cmntcount = cmntmap.reduceByKey(lambda x,y: x+y)
cmntcount.collect()

[('Aliyar', 5), ('Dumdar', 5), ('Ertugul', 6)]

##### Count the number of UserNames starting with each English alphabet.

In [26]:
names = Data.map(lambda line: line.split(',')[0]).distinct()
names.collect()

['Aliyar', 'Dumdar', 'Ertugul']

In [27]:
alphamap = names.map(lambda name: (name[0], 1))
alphacount = alphamap.reduceByKey(lambda x,y: x+y)
alphacount.collect()

[('A', 1), ('D', 1), ('E', 1)]

##### Write custom partitioner to partition the data on the basis of first letter of user name.

In [28]:
alphacomments = Data.map(lambda line: (line[0], line))
partioned = alphacomments.groupByKey().map(lambda x : (x[0], list(x[1])))
partioned.collect()

[('A',
  ["Aliyar, This is my first pair of 5-pocket jeans I've purchased from Dickies I typically wear 505s in 36x32 for everyday wear I used to work construction and religiously",
   'Aliyar, I decided to order the Indigo Blue Rough in 36x34 When the jeans first arrived they were very stiff and looked huge on me Too wide in the waist and way too ',
   "Aliyar, In the future I'd probably order 36x32 instead of 36x34 I don't like having to worry about my jeans shrinking so I always buy on the larger side and warm wash and high dry",
   'Aliyar, Very nice color nice deep dark indigo Very similar to the 501 shrink-to-fit',
   'Aliyar, The leg opening is about 1" wider than my 505s Not as slim as my 505s but definitely not in the baggy category I\'d definitely buy again']),
 ('D',
  ['Dumdar, wore the Carpenter Duck jeans in 36x34',
   'Dumdar, long in the length But after a hot wash and high heat dry they fit much better The waist is perfect I always wear a belt Still about 15" too long 

##### Sort the data on the basis of the length of the comment given by each user.

In [29]:
Data.sortBy(lambda line: len(line.split(',')[1]), ascending=False).collect()

["Ertugul, Originally got stone washed for daily wear and indigo for a little more formal (in a jeans-sense) but have since found that indigo becomes a nice stone washed after a few months so now that's what I get and simply let them fade over time",
 "Ertugul, The most comfortable jeans I've ever had out of the package were a pair from LLBean that felt like super soft cotton but they were on sale and since have only been available ",
 "Aliyar, In the future I'd probably order 36x32 instead of 36x34 I don't like having to worry about my jeans shrinking so I always buy on the larger side and warm wash and high dry",
 'Dumdar, couple from a few years ago have bleach splash spots from cleaning stuff and yard and car (and motorcycle) stains at the knees I now have 6 pair and wear them every day ',
 "Aliyar, This is my first pair of 5-pocket jeans I've purchased from Dickies I typically wear 505s in 36x32 for everyday wear I used to work construction and religiously",
 'Dumdar, long in the 

##### Find the user who have given maximum number of comments

In [33]:
usrcmnt = Data.map(lambda line: (line.split(',')[0], 1))
usrcmnt.reduceByKey(lambda x,y: x+y).top(1, key=lambda x: x[1])[0][0]

'Ertugul'

##### remove stop words from the comments of the users in the above dataset.

In [5]:
stopWords = sc.textFile("stop_words_english.txt").map(lambda x: x)
broadcastWords = sc.broadcast(stopWords.collect())

In [6]:
wordsWithoutStopWords = Data.map(lambda x: list(set(x.split(',')[1].split(' ')).difference(set(broadcastWords.value)))[1:])
wordsWithoutStopWords.collect()

[['jeans',
  'work',
  'typically',
  'stiff',
  'construction',
  'This',
  '36x32',
  'pair',
  'religiously',
  'wear',
  'Dickies',
  '5-pocket',
  'purchased',
  '505s',
  "I've",
  'everyday'],
 ['jeans', 'Duck', 'stiff', 'Carpenter', '36x34', 'wore'],
 ['jeans',
  'Too',
  'stiff',
  'huge',
  'Indigo',
  'waist',
  'looked',
  'wide',
  'arrived',
  'When',
  'Blue',
  '36x34',
  'order',
  'decided',
  'Rough'],
 ['wash',
  'stiff',
  'perfect',
  'waist',
  'Still',
  'length',
  'belt',
  'fit',
  'wear',
  'hot',
  'dry',
  '15"',
  'But',
  'heat',
  'high',
  'long',
  'The'],
 ['hem', 'Nothing', 'error', 'ordering'],
 ['jeans',
  'wash',
  'worry',
  'larger',
  'stiff',
  'buy',
  "I'd",
  '36x32',
  'warm',
  '36x34',
  'order',
  'dry',
  'high',
  'In',
  'shrinking',
  'future'],
 ['Very', 'shrink-to-fit', 'deep', 'dark', 'indigo', '501', 'color', 'nice'],
 ['leg',
  'opening',
  'buy',
  "I'd",
  '1"',
  'Not',
  'wider',
  'category',
  'slim',
  '505s',
  'The',


##### generate co-occurring words that co-occurred more than 5 times in the comments

In [7]:
import itertools

In [8]:
rddx = wordsWithoutStopWords.map(lambda x: [(combination, 1) for i in range(2,len(x)+1) for combination in itertools.combinations(x, i)])

In [9]:
rddy = rddx.flatMap(lambda x: x).reduceByKey(lambda x,y: x+y)

In [12]:
rddz = rddy.sortBy(lambda a: a[1], ascending=False).filter(lambda x: x[1]>5)

In [13]:
rddz.collect()

[(('jeans', 'stiff'), 8)]

# The End